In [ ]:
import sys; sys.dont_write_bytecode=1
import warnings
from typing import Literal as Lit
import itertools as it
from numpy import nan, log, exp
import numpy as np
import pandas as pd
import deps as _
import Common
from importlib import reload; reload(_)

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
TICKERS     = _.BEST_20
TOP         = _.TOP_500
TF, SFX, SRC, UNTIL, GRAN, ANUM = _.PRESET('Y')
PRDS, SLGS  = _.aPrds(TF, [1,2,3,4,5,6]).tolist(), _.aSlug(TF, [1,2,3,4,5,6]).tolist()


# LI_RNK_1 = Common.LI_RNK_1[TF]
# LI_RNK_2 = Common.LI_RNK_2[TF]
TP_RNK_1 = Common.TP_RNK_1[TF]
TP_RNK_2 = Common.TP_RNK_2[TF]

RNK_1:TP_RNK_1  = 'High Cor 1Y' 
RNK_2:TP_RNK_2  = 'High Gap 1Y'
IND_1, IND_2    = _.np_remove_prefix([RNK_1, RNK_2])
WGHTS           = [1,1]
NORM_WGHTS      = True
RCOMB           = 'High Art'


DOWNLOAD, COMPUTE =  True, True
FULL_CLOCK        =  True
SYNC:_.TP_SYNC    = 'SAVE'
DIRECTORY         = 'Storage/'
RESULTS           = 'Results/'
FORMAT            = 'parquet'

# Section 1

In [ ]:
def GET_HIST_DATA(TICKERS, SRC, UNTIL, start=None):
    
    Raw = _.yf_api_hdatas(TICKERS, interval=SRC, period=UNTIL, start=start)

    Src = pd.DataFrame()
    Src['N']      = Raw['N']
    Src['Ticker'] = Raw['Ticker']
    Src['Date']   = Raw['Date']
    Src['Price']  = Raw['Adj Close']
    return Src

Src = GET_HIST_DATA(TICKERS, SRC, UNTIL, start=None)
Src

In [ ]:
def MAKE_PAIRS_KEYS(Src):
    Pairs = _.pd_make_pairs(Src['Ticker'], unique=1, prefix='Ticker.')
    Date  = Src[['Date']].drop_duplicates().sort_values('Date', ascending=False) . reset_index(drop=1)
    return pd.merge(Pairs, Date, 'cross')

Keys = MAKE_PAIRS_KEYS(Src)
Keys

In [ ]:
def MAKE_PAIRS_SOURCE(Keys, Src, TF, FULL_CLOCK):
    
    Df = Keys.copy()
    Df['Trans']   = _.in_transit(Df, 'Date', TF, FULL_CLOCK)
    Df['Price.X'] = _.pd_relate(Keys, ['Ticker.X','Date'], Src, ['Ticker','Date'], 'Price') .add_suffix('.X')
    Df['Price.Y'] = _.pd_relate(Keys, ['Ticker.Y','Date'], Src, ['Ticker','Date'], 'Price') .add_suffix('.Y')
    Df = Df.dropna().reset_index(drop=1)
    
    Df.insert(0, 'N', _.Nth(Keys, 'Date', By='Ticker.XY', sign=(-1), asc=0, adj=(+1)))

    Df['LnPrc.X'] = _.log(Df['Price.X'])
    Df['LnPrc.Y'] = _.log(Df['Price.Y'])
    return Df

Src2 = MAKE_PAIRS_SOURCE(Keys, Src, TF, FULL_CLOCK)
Src2

# Section 2

In [6]:
def CALCULATE_PRC_STATS(Src, K, PRDS, SLGS):
    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS

    Df = Src.loc[:, None:'LnPrc.Y'].copy()
    
    def _correlate(win, minWin):
        return _.pd_rollby_correlate('pandas',  Df,'LnPrc.Y','LnPrc.X',  By='Ticker.XY', win=win, minWin=minWin,  stp=(-1), _mlt=100, _rnd=1)

    Df['Cor']       = ''
    Df[f'Cor {S1}'] = _correlate(win=P1*K, minWin=P1*K)  # nan
    Df[f'Cor {S2}'] = _correlate(win=P2*K, minWin=P1*K)  # nan
    Df[f'Cor {S3}'] = _correlate(win=P3*K, minWin=P2*K)  # nan
    Df[f'Cor {S4}'] = _correlate(win=P4*K, minWin=P3*K)  # nan
    Df[f'Cor {S5}'] = _correlate(win=P5*K, minWin=P4*K)  # nan
    Df[f'Cor {S6}'] = _correlate(win=P6*K, minWin=P5*K)  # nan
    return Df


PSts = CALCULATE_PRC_STATS(Src2, GRAN, PRDS, SLGS)
PSts

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,Price.X,Price.Y,LnPrc.X,LnPrc.Y,Cor,Cor 1Y,Cor 3Y,Cor 5Y,Cor 10Y,Cor 20Y,Cor 40Y
0,0,AAPL,MSFT,AAPL-MSFT,2024-11-01,0,224.229996,423.029999,5.412672,6.047443,,48.2,88.6,93.9,98.1,93.1,80.3
1,-1,AAPL,MSFT,AAPL-MSFT,2024-10-01,0,225.661728,406.350006,5.419037,6.007215,,46.1,87.8,94.1,98.1,92.9,80.2
2,-2,AAPL,MSFT,AAPL-MSFT,2024-09-01,0,232.743942,430.299988,5.449939,6.064483,,58.2,86.1,94.4,98.1,92.7,80.1
3,-3,AAPL,MSFT,AAPL-MSFT,2024-08-01,0,228.483871,416.389496,5.431466,6.031621,,57.8,85.1,94.6,98.0,92.3,79.9
4,-4,AAPL,MSFT,AAPL-MSFT,2024-07-01,0,221.579468,417.597321,5.400781,6.034518,,50.3,84.3,94.6,98.0,92.0,79.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48241,-341,EPAM,MKTX,EPAM-MKTX,2012-07-01,0,16.040001,27.098576,2.775086,3.299481,,NaN,NaN,NaN,NaN,NaN,NaN
48242,-342,EPAM,MKTX,EPAM-MKTX,2012-06-01,0,16.990000,23.888359,2.832625,3.173391,,NaN,NaN,NaN,NaN,NaN,NaN
48243,-343,EPAM,MKTX,EPAM-MKTX,2012-05-01,0,17.500000,28.857672,2.862201,3.362376,,NaN,NaN,NaN,NaN,NaN,NaN
48244,-344,EPAM,MKTX,EPAM-MKTX,2012-04-01,0,20.639999,30.662949,3.027231,3.423055,,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def CALCULATE_VARIATIONS(Src, K, PRDS, SLGS):
    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS
    
    Df = Src.copy()

    def _Return(Val, N):
        return _.Return(Df, Val, By='Ticker.XY', N=N, stp=(-1), mult=100, rnd=2)

    Df['Vars.X+']       = ''
    Df[f'Var.X +{S1}']  = _Return('Price.X', N=P1*K)
    Df[f'Var.X +{S2}']  = _Return('Price.X', N=P2*K)
    Df[f'Var.X +{S3}']  = _Return('Price.X', N=P3*K)
    Df[f'Var.X +{S4}']  = _Return('Price.X', N=P4*K)
    Df[f'Var.X +{S5}']  = _Return('Price.X', N=P5*K)
    Df[f'Var.X +{S6}']  = _Return('Price.X', N=P6*K)

    Df['Vars.Y+']       = ''
    Df[f'Var.Y +{S1}']  = _Return('Price.Y', N=P1*K)
    Df[f'Var.Y +{S2}']  = _Return('Price.Y', N=P2*K)
    Df[f'Var.Y +{S3}']  = _Return('Price.Y', N=P3*K)
    Df[f'Var.Y +{S4}']  = _Return('Price.Y', N=P4*K)
    Df[f'Var.Y +{S5}']  = _Return('Price.Y', N=P5*K)
    Df[f'Var.Y +{S6}']  = _Return('Price.Y', N=P6*K)

    def _Variat(Val, N):
        return _.Variat(Df, Val, By='Ticker.XY', N=N, stp=(-1), mult=100, rnd=2)

    Df['Vars.X-']       = ''
    Df[f'Var.X -{S1}']  = _Variat('Price.X', N=P1*K)
    Df[f'Var.X -{S2}']  = _Variat('Price.X', N=P2*K)
    Df[f'Var.X -{S3}']  = _Variat('Price.X', N=P3*K)
    Df[f'Var.X -{S4}']  = _Variat('Price.X', N=P4*K)
    Df[f'Var.X -{S5}']  = _Variat('Price.X', N=P5*K)
    Df[f'Var.X -{S6}']  = _Variat('Price.X', N=P6*K)

    Df['Vars.Y-']       = ''
    Df[f'Var.Y -{S1}']  = _Variat('Price.Y', N=P1*K)
    Df[f'Var.Y -{S2}']  = _Variat('Price.Y', N=P2*K)
    Df[f'Var.Y -{S3}']  = _Variat('Price.Y', N=P3*K)
    Df[f'Var.Y -{S4}']  = _Variat('Price.Y', N=P4*K)
    Df[f'Var.Y -{S5}']  = _Variat('Price.Y', N=P5*K)
    Df[f'Var.Y -{S6}']  = _Variat('Price.Y', N=P6*K)

    def _pct_point(Y, X):
        return _.pct_point(Df, Y, X, geo=['X','Y','Z'], pct=['X','Y','Z'], rnd=1)

    Df['Gap']       = ''
    Df[f'Gap {S1}'] = _pct_point(f'Var.Y -{S1}',  f'Var.X -{S1}')
    Df[f'Gap {S2}'] = _pct_point(f'Var.Y -{S2}',  f'Var.X -{S2}')
    Df[f'Gap {S3}'] = _pct_point(f'Var.Y -{S3}',  f'Var.X -{S3}')
    Df[f'Gap {S4}'] = _pct_point(f'Var.Y -{S4}',  f'Var.X -{S4}')
    Df[f'Gap {S5}'] = _pct_point(f'Var.Y -{S5}',  f'Var.X -{S5}')
    Df[f'Gap {S6}'] = _pct_point(f'Var.Y -{S6}',  f'Var.X -{S6}')

    return Df


Vars = CALCULATE_VARIATIONS(Src2, GRAN, PRDS, SLGS)
Vars

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,Price.X,Price.Y,LnPrc.X,LnPrc.Y,Vars.X+,Var.X +1Y,Var.X +3Y,Var.X +5Y,Var.X +10Y,Var.X +20Y,Var.X +40Y,Vars.Y+,Var.Y +1Y,Var.Y +3Y,Var.Y +5Y,Var.Y +10Y,Var.Y +20Y,Var.Y +40Y,Vars.X-,Var.X -1Y,Var.X -3Y,Var.X -5Y,Var.X -10Y,Var.X -20Y,Var.X -40Y,Vars.Y-,Var.Y -1Y,Var.Y -3Y,Var.Y -5Y,Var.Y -10Y,Var.Y -20Y,Var.Y -40Y,Gap,Gap 1Y,Gap 3Y,Gap 5Y,Gap 10Y,Gap 20Y,Gap 40Y
0,0,AAPL,MSFT,AAPL-MSFT,2024-11-01,0,224.229996,423.029999,5.412672,6.047443,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,18.78,38.07,247.21,750.30,22101.06,NaN,,12.48,31.26,192.57,934.85,2429.14,NaN,,-5.3,-4.9,-15.7,21.7,-88.6,NaN
1,-1,AAPL,MSFT,AAPL-MSFT,2024-10-01,0,225.661728,406.350006,5.419037,6.007215,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,32.97,53.33,275.40,842.33,28489.43,NaN,,21.08,25.69,196.74,912.25,2228.66,NaN,,-8.9,-18.0,-21.0,7.4,-91.9,NaN
2,-2,AAPL,MSFT,AAPL-MSFT,2024-09-01,0,232.743942,430.299988,5.449939,6.064483,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,36.79,67.42,330.04,941.85,39773.59,NaN,,37.30,56.56,224.03,985.56,2394.44,NaN,,0.4,-6.5,-24.7,4.2,-93.7,NaN
3,-3,AAPL,MSFT,AAPL-MSFT,2024-08-01,0,228.483871,416.389496,5.431466,6.031621,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,22.54,53.40,354.69,910.31,43878.55,NaN,,28.27,41.76,217.27,978.66,2351.96,NaN,,4.7,-7.6,-30.2,6.8,-94.4,NaN
4,-4,AAPL,MSFT,AAPL-MSFT,2024-07-01,0,221.579468,417.597321,5.400781,6.034518,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,13.65,54.86,332.05,950.50,45384.99,NaN,,25.51,50.63,221.91,1038.69,2256.36,NaN,,10.4,-2.7,-25.5,8.4,-94.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48241,-341,EPAM,MKTX,EPAM-MKTX,2012-07-01,0,16.040001,27.098576,2.775086,3.299481,,80.49,362.03,435.72,2077.37,NaN,NaN,,73.37,234.66,605.02,873.79,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
48242,-342,EPAM,MKTX,EPAM-MKTX,2012-06-01,0,16.990000,23.888359,2.832625,3.173391,,59.98,319.25,394.94,1635.02,NaN,NaN,,77.84,260.10,692.71,944.39,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
48243,-343,EPAM,MKTX,EPAM-MKTX,2012-05-01,0,17.500000,28.857672,2.862201,3.362376,,32.34,310.91,379.31,1834.40,NaN,NaN,,36.13,183.54,520.77,848.69,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
48244,-344,EPAM,MKTX,EPAM-MKTX,2012-04-01,0,20.639999,30.662949,3.027231,3.423055,,4.17,213.52,273.06,1183.87,NaN,NaN,,25.04,159.01,490.17,735.56,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def CALCULATE_VAR_STATS(Src, Vars, K, PRDS, SLGS):
    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS

    Df = Src.copy()


    def _roll_stats(algo, Val, win, minWin):
        return _.pd_roll_calc(algo, Vars, Val, By='Ticker.XY', win=win, minWin=minWin, stp=(-1), _in='p100', _out='m100', fig=1)

    def add_roll_stats(Name, algo, P, S):
        Df[f'Gap.{P} {Name}.{P1}'] = _roll_stats(algo, f'Gap {S}', win=P1*K, minWin=P1*K)
        Df[f'Gap.{P} {Name}.{P2}'] = _roll_stats(algo, f'Gap {S}', win=P2*K, minWin=P1*K)
        Df[f'Gap.{P} {Name}.{P3}'] = _roll_stats(algo, f'Gap {S}', win=P3*K, minWin=P2*K)
        Df[f'Gap.{P} {Name}.{P4}'] = _roll_stats(algo, f'Gap {S}', win=P4*K, minWin=P3*K)
        Df[f'Gap.{P} {Name}.{P5}'] = _roll_stats(algo, f'Gap {S}', win=P5*K, minWin=P4*K)
        Df[f'Gap.{P} {Name}.{P6}'] = _roll_stats(algo, f'Gap {S}', win=P6*K, minWin=P5*K)
        pass
    
    Df['Gap Avg'] = ''
    add_roll_stats('Avg', 'geo_mean', P1, S1)
    add_roll_stats('Avg', 'geo_mean', P2, S2)
    add_roll_stats('Avg', 'geo_mean', P3, S3)
    add_roll_stats('Avg', 'geo_mean', P4, S4)
    add_roll_stats('Avg', 'geo_mean', P5, S5)
    add_roll_stats('Avg', 'geo_mean', P6, S6)

    Df['Gap Dev'] = ''
    add_roll_stats('Dev', 'geo_std', P1, S1)
    add_roll_stats('Dev', 'geo_std', P2, S2)
    add_roll_stats('Dev', 'geo_std', P3, S3)
    add_roll_stats('Dev', 'geo_std', P4, S4)
    add_roll_stats('Dev', 'geo_std', P5, S5)
    add_roll_stats('Dev', 'geo_std', P6, S6)


    def _ZLevel(Val, Avg, Dev):
        return _.ZLevel(..., Vars[Val], Df[Avg], Df[Dev], Pct=[1,1,1], Log=[1,1,1], fig=3) 

    def add_ZLevel(P, S):
        Df[f'Gap.{P} Lvl.{P1}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P1}',  f'Gap.{P} Dev.{P1}')
        Df[f'Gap.{P} Lvl.{P2}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P2}',  f'Gap.{P} Dev.{P2}')
        Df[f'Gap.{P} Lvl.{P3}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P3}',  f'Gap.{P} Dev.{P3}')
        Df[f'Gap.{P} Lvl.{P4}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P4}',  f'Gap.{P} Dev.{P4}')
        Df[f'Gap.{P} Lvl.{P5}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P5}',  f'Gap.{P} Dev.{P5}')
        Df[f'Gap.{P} Lvl.{P6}'] = _ZLevel(f'Gap {S}',  f'Gap.{P} Avg.{P6}',  f'Gap.{P} Dev.{P6}')
        pass

    Df['Gap Lvl'] = ''
    add_ZLevel(P1, S1)
    add_ZLevel(P2, S2)
    add_ZLevel(P3, S3)
    add_ZLevel(P4, S4)
    add_ZLevel(P5, S5)
    add_ZLevel(P6, S6)

    return Df


VSts = CALCULATE_VAR_STATS(Src2, Vars, GRAN, PRDS, SLGS)
VSts

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,Price.X,Price.Y,LnPrc.X,LnPrc.Y,Gap Avg,Gap.1 Avg.1,Gap.1 Avg.3,Gap.1 Avg.5,Gap.1 Avg.10,Gap.1 Avg.20,Gap.1 Avg.40,Gap.3 Avg.1,Gap.3 Avg.3,Gap.3 Avg.5,Gap.3 Avg.10,Gap.3 Avg.20,Gap.3 Avg.40,Gap.5 Avg.1,Gap.5 Avg.3,Gap.5 Avg.5,Gap.5 Avg.10,Gap.5 Avg.20,Gap.5 Avg.40,Gap.10 Avg.1,Gap.10 Avg.3,Gap.10 Avg.5,Gap.10 Avg.10,Gap.10 Avg.20,Gap.10 Avg.40,Gap.20 Avg.1,Gap.20 Avg.3,Gap.20 Avg.5,Gap.20 Avg.10,Gap.20 Avg.20,Gap.20 Avg.40,Gap.40 Avg.1,Gap.40 Avg.3,Gap.40 Avg.5,Gap.40 Avg.10,Gap.40 Avg.20,Gap.40 Avg.40,Gap Dev,Gap.1 Dev.1,Gap.1 Dev.3,Gap.1 Dev.5,Gap.1 Dev.10,Gap.1 Dev.20,Gap.1 Dev.40,Gap.3 Dev.1,Gap.3 Dev.3,Gap.3 Dev.5,Gap.3 Dev.10,Gap.3 Dev.20,Gap.3 Dev.40,Gap.5 Dev.1,Gap.5 Dev.3,Gap.5 Dev.5,Gap.5 Dev.10,Gap.5 Dev.20,Gap.5 Dev.40,Gap.10 Dev.1,Gap.10 Dev.3,Gap.10 Dev.5,Gap.10 Dev.10,Gap.10 Dev.20,Gap.10 Dev.40,Gap.20 Dev.1,Gap.20 Dev.3,Gap.20 Dev.5,Gap.20 Dev.10,Gap.20 Dev.20,Gap.20 Dev.40,Gap.40 Dev.1,Gap.40 Dev.3,Gap.40 Dev.5,Gap.40 Dev.10,Gap.40 Dev.20,Gap.40 Dev.40,Gap Lvl,Gap.1 Lvl.1,Gap.1 Lvl.3,Gap.1 Lvl.5,Gap.1 Lvl.10,Gap.1 Lvl.20,Gap.1 Lvl.40,Gap.3 Lvl.1,Gap.3 Lvl.3,Gap.3 Lvl.5,Gap.3 Lvl.10,Gap.3 Lvl.20,Gap.3 Lvl.40,Gap.5 Lvl.1,Gap.5 Lvl.3,Gap.5 Lvl.5,Gap.5 Lvl.10,Gap.5 Lvl.20,Gap.5 Lvl.40,Gap.10 Lvl.1,Gap.10 Lvl.3,Gap.10 Lvl.5,Gap.10 Lvl.10,Gap.10 Lvl.20,Gap.10 Lvl.40,Gap.20 Lvl.1,Gap.20 Lvl.3,Gap.20 Lvl.5,Gap.20 Lvl.10,Gap.20 Lvl.20,Gap.20 Lvl.40,Gap.40 Lvl.1,Gap.40 Lvl.3,Gap.40 Lvl.5,Gap.40 Lvl.10,Gap.40 Lvl.20,Gap.40 Lvl.40
0,0,AAPL,MSFT,AAPL-MSFT,2024-11-01,0,224.229996,423.029999,5.412672,6.047443,,13.5,2.1,-4.0,1.3,-13.3,3.1,6.3,-13.8,-8.7,3.5,-38.9,7.8,-18.6,-14.1,-3.9,1.0,-55.5,10.0,14.0,12.3,2.1,-44.8,-78.7,-8.3,-94.2,-96.2,-96.1,-95.5,-77.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,15.0,16.2,18.3,22.4,38.7,56.2,15.3,24.2,22.8,25.3,99.2,171.5,10.5,10.3,25.7,37.2,161.3,343.9,7.1,15.7,23.3,117.4,304.6,1212.1,33.4,47.4,50.9,78.7,673.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1.296,-0.501,-0.081,-0.333,0.270,-0.191,-0.782,0.453,0.199,-0.375,0.642,-0.126,0.351,-0.192,-0.573,-0.571,0.665,-0.179,0.953,0.551,0.838,1.018,1.247,0.110,2.345,2.832,2.607,1.601,-0.341,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1,AAPL,MSFT,AAPL-MSFT,2024-10-01,0,225.661728,406.350006,5.419037,6.007215,,15.5,2.5,-4.1,1.2,-13.6,3.1,7.8,-14.2,-8.6,3.5,-39.2,7.8,-18.9,-13.9,-3.1,0.5,-55.7,10.1,12.9,12.1,0.9,-46.0,-78.7,-8.4,-94.7,-96.4,-96.1,-95.5,-77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,13.6,16.2,18.3,22.5,39.5,56.3,14.9,24.1,22.9,25.4,99.5,171.8,10.4,10.3,26.4,38.5,161.1,344.7,6.9,15.7,24.4,120.3,305.7,1216.9,24.7,40.4,47.8,80.0,676.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1.861,-0.785,-0.306,-0.518,0.159,-0.277,-1.970,-0.210,-0.526,-1.029,0.433,-0.274,-0.265,-0.878,-0.872,-0.739,0.603,-0.222,-0.748,-0.294,0.286,0.871,1.155,0.062,1.922,2.390,1.871,1.000,-0.509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2,AAPL,MSFT,AAPL-MSFT,2024-09-01,0,232.743942,430.299988,5.449939,6.064483,,19.1,3.3,-3.7,1.2,-13.9,3.1,10.2,-13.4,-8.1,3.6,-39.4,7.9,-17.1,-13.2,-2.2,-0.0,-55.8,10.2,13.0,12.7,-0.0,-47.2,-78.6,-8.4,-94.9,-96.5,-96.2,-95.4,-76.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,11.4,16.3,18.5,22.5,39.9,56.4,11.5,24.8,23.0,25.3,99.7,172.1,12.7,10.9,26.7,39.5,160.9,345.5,6.9,15.8,25.4,124.1,308.2,1221.9,20.5,36.4,46.2,82.8,679.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1.582,-0.189,0.246,-0.039,0.458,-0.059,-1.510,0.346,0.083,-0.455,0.627,-0.143,-0.804,-1.374,-1.105,-0.852,0.556,-0.255,-1.215,-0.535,0.182,0.842,1.125,0.050,1.133,1.894,1.331,0.521,-0.633,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-3,AAPL,MSFT,AAPL-MSFT,2024-08-01,0,228.483871,416.389496,5.431466,6.031621,,20.0,3.6,-3.4,1.2,-14.1,3.1,11.1,-13.3,-7.7,3.7,-39.6,7.9,-15.7,-12.5,-1.2,-0.3,-55.8,10.3,12.4,13.0,-0.7,-48.4,-78.5,-8.4,-95.2,-96.5,-96.2,-95.4,-76.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,10.2,16.4,18.9,22.5,40.0,56.4,10.5,24.9,23.4,25.2,99.6,172.4,12.6,10.8,26.9,40.1,160.8,346.2,7.7,15.8,26.0,129.0,311.4,1226.9,23.2,34.4,45.4,85.5,681.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,-1.404,0.070,0.465,0.168,0.588,0.034,-1.846,0.286,0.005,-0.513,0.615,-0.155,-1.591,-2

In [ ]:
def LONG_SHORT_RETURNS(Src, Vars, PRDS, SLGS):
    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS
    
    Df = pd.DataFrame()

    # Df['N']         = Src2['N'] 
    # Df['Date']      = Src2['Date'] 
    # Df[' '] = ''
    # Df['Ticker.Y']  = Src2['Ticker.Y']
    # Df['Ticker.X']  = Src2['Ticker.X']
    # Df['  '] = ''
    # Df['Var.Y 1Y']  = Vars['Var.Y -1Y']
    # Df['Gap 1Y']    = Vars['Gap 1Y']
    # Df['Var.X 1Y']  = Vars['Var.X -1Y']
    # Df['   '] = ''

    is_ngtv_1, is_pstv_1 = (Vars[f'Gap {S1}'] < 0), (0 < Vars[f'Gap {S1}'])
    is_ngtv_2, is_pstv_2 = (Vars[f'Gap {S2}'] < 0), (0 < Vars[f'Gap {S2}'])
    is_ngtv_3, is_pstv_3 = (Vars[f'Gap {S3}'] < 0), (0 < Vars[f'Gap {S3}'])
    is_ngtv_4, is_pstv_4 = (Vars[f'Gap {S4}'] < 0), (0 < Vars[f'Gap {S4}'])
    is_ngtv_5, is_pstv_5 = (Vars[f'Gap {S5}'] < 0), (0 < Vars[f'Gap {S5}'])
    is_ngtv_6, is_pstv_6 = (Vars[f'Gap {S6}'] < 0), (0 < Vars[f'Gap {S6}'])

    Df['Long Short']  = ''
    Df[f'Long {S1}']  = np.select([is_ngtv_1, is_pstv_1], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S1}'] = np.select([is_ngtv_1, is_pstv_1], [Src['Ticker.X'], Src['Ticker.Y']], '')

    Df[f'Long {S2}']  = np.select([is_ngtv_2, is_pstv_2], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S2}'] = np.select([is_ngtv_2, is_pstv_2], [Src['Ticker.X'], Src['Ticker.Y']], '')

    Df[f'Long {S3}']  = np.select([is_ngtv_3, is_pstv_3], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S3}'] = np.select([is_ngtv_3, is_pstv_3], [Src['Ticker.X'], Src['Ticker.Y']], '')

    Df[f'Long {S4}']  = np.select([is_ngtv_4, is_pstv_4], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S4}'] = np.select([is_ngtv_4, is_pstv_4], [Src['Ticker.X'], Src['Ticker.Y']], '')

    Df[f'Long {S5}']  = np.select([is_ngtv_5, is_pstv_5], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S5}'] = np.select([is_ngtv_5, is_pstv_5], [Src['Ticker.X'], Src['Ticker.Y']], '')

    Df[f'Long {S6}']  = np.select([is_ngtv_6, is_pstv_6], [Src['Ticker.Y'], Src['Ticker.X']], '')
    Df[f'Short {S6}'] = np.select([is_ngtv_6, is_pstv_6], [Src['Ticker.X'], Src['Ticker.Y']], '')
    
    Df['LS Vars']          = ''
    Df[f'Long Var +{S1}']  = np.select([is_ngtv_1, is_pstv_1], [Vars[f'Var.Y +{S1}'], Vars[f'Var.X +{S1}']], nan)
    Df[f'Short Var +{S1}'] = np.select([is_ngtv_1, is_pstv_1], [Vars[f'Var.X +{S1}'], Vars[f'Var.Y +{S1}']], nan)

    Df[f'Long Var +{S2}']  = np.select([is_ngtv_2, is_pstv_2], [Vars[f'Var.Y +{S2}'], Vars[f'Var.X +{S2}']], nan)
    Df[f'Short Var +{S2}'] = np.select([is_ngtv_2, is_pstv_2], [Vars[f'Var.X +{S2}'], Vars[f'Var.Y +{S2}']], nan)

    Df[f'Long Var +{S3}']  = np.select([is_ngtv_3, is_pstv_3], [Vars[f'Var.Y +{S3}'], Vars[f'Var.X +{S3}']], nan)
    Df[f'Short Var +{S3}'] = np.select([is_ngtv_3, is_pstv_3], [Vars[f'Var.X +{S3}'], Vars[f'Var.Y +{S3}']], nan)

    Df[f'Long Var +{S4}']  = np.select([is_ngtv_4, is_pstv_4], [Vars[f'Var.Y +{S4}'], Vars[f'Var.X +{S4}']], nan)
    Df[f'Short Var +{S4}'] = np.select([is_ngtv_4, is_pstv_4], [Vars[f'Var.X +{S4}'], Vars[f'Var.Y +{S4}']], nan)

    Df[f'Long Var +{S5}']  = np.select([is_ngtv_5, is_pstv_5], [Vars[f'Var.Y +{S5}'], Vars[f'Var.X +{S5}']], nan)
    Df[f'Short Var +{S5}'] = np.select([is_ngtv_5, is_pstv_5], [Vars[f'Var.X +{S5}'], Vars[f'Var.Y +{S5}']], nan)

    Df[f'Long Var +{S6}']  = np.select([is_ngtv_6, is_pstv_6], [Vars[f'Var.Y +{S6}'], Vars[f'Var.X +{S6}']], nan)
    Df[f'Short Var +{S6}'] = np.select([is_ngtv_6, is_pstv_6], [Vars[f'Var.X +{S6}'], Vars[f'Var.Y +{S6}']], nan)

    Df['LS Rtn']        = ''
    Df[f'LS Rtn +{S1}'] = ((Df[f'Long Var +{S1}'] - Df[f'Short Var +{S1}']) /2) .round(2)
    Df[f'LS Rtn +{S2}'] = ((Df[f'Long Var +{S2}'] - Df[f'Short Var +{S2}']) /2) .round(2)
    Df[f'LS Rtn +{S3}'] = ((Df[f'Long Var +{S3}'] - Df[f'Short Var +{S3}']) /2) .round(2)
    Df[f'LS Rtn +{S4}'] = ((Df[f'Long Var +{S4}'] - Df[f'Short Var +{S4}']) /2) .round(2)
    Df[f'LS Rtn +{S5}'] = ((Df[f'Long Var +{S5}'] - Df[f'Short Var +{S5}']) /2) .round(2)
    Df[f'LS Rtn +{S6}'] = ((Df[f'Long Var +{S6}'] - Df[f'Short Var +{S6}']) /2) .round(2)

    return Df


LS = LONG_SHORT_RETURNS(Src2, Vars, PRDS, SLGS)
LS

,Long Short,Long 1Y,Short 1Y,Long 3Y,Short 3Y,Long 5Y,Short 5Y,Long 10Y,Short 10Y,Long 20Y,Short 20Y,Long 40Y,Short 40Y,LS Vars,Long Var +1Y,Short Var +1Y,Long Var +3Y,Short Var +3Y,Long Var +5Y,Short Var +5Y,Long Var +10Y,Short Var +10Y,Long Var +20Y,Short Var +20Y,Long Var +40Y,Short Var +40Y,LS Rtn,LS Rtn +1Y,LS Rtn +3Y,LS Rtn +5Y,LS Rtn +10Y,LS Rtn +20Y,LS Rtn +40Y
0,NaN,MSFT,AAPL,MSFT,AAPL,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,MSFT,AAPL,MSFT,AAPL,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,AAPL,MSFT,MSFT,AAPL,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,AAPL,MSFT,MSFT,AAPL,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,AAPL,MSFT,MSFT,AAPL,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,AAPL,MSFT,AAPL,MSFT,MSFT,AAPL,AAPL,MSFT,MSFT,AAPL,,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
def CALCULATE_ABSOLUTES(Vars, VSts):
    Inds = pd.concat([ Vars.loc[:, 'Gap':None], VSts.loc[:, 'Gap Avg':None] ], axis=1)
    IAbs = Inds.map(pd.to_numeric, errors='coerce').abs().add_prefix('Abs ')
    return IAbs

IAbs = CALCULATE_ABSOLUTES(Vars, VSts)
IAbs

,Abs Gap,Abs Gap 1Y,Abs Gap 3Y,Abs Gap 5Y,Abs Gap 10Y,Abs Gap 20Y,Abs Gap 40Y,Abs Gap Avg,Abs Gap.1 Avg.1,Abs Gap.1 Avg.3,Abs Gap.1 Avg.5,Abs Gap.1 Avg.10,Abs Gap.1 Avg.20,Abs Gap.1 Avg.40,Abs Gap.3 Avg.1,Abs Gap.3 Avg.3,Abs Gap.3 Avg.5,Abs Gap.3 Avg.10,Abs Gap.3 Avg.20,Abs Gap.3 Avg.40,Abs Gap.5 Avg.1,Abs Gap.5 Avg.3,Abs Gap.5 Avg.5,Abs Gap.5 Avg.10,Abs Gap.5 Avg.20,Abs Gap.5 Avg.40,Abs Gap.10 Avg.1,Abs Gap.10 Avg.3,Abs Gap.10 Avg.5,Abs Gap.10 Avg.10,Abs Gap.10 Avg.20,Abs Gap.10 Avg.40,Abs Gap.20 Avg.1,Abs Gap.20 Avg.3,Abs Gap.20 Avg.5,Abs Gap.20 Avg.10,Abs Gap.20 Avg.20,Abs Gap.20 Avg.40,Abs Gap.40 Avg.1,Abs Gap.40 Avg.3,Abs Gap.40 Avg.5,Abs Gap.40 Avg.10,Abs Gap.40 Avg.20,Abs Gap.40 Avg.40,Abs Gap Dev,Abs Gap.1 Dev.1,Abs Gap.1 Dev.3,Abs Gap.1 Dev.5,Abs Gap.1 Dev.10,Abs Gap.1 Dev.20,Abs Gap.1 Dev.40,Abs Gap.3 Dev.1,Abs Gap.3 Dev.3,Abs Gap.3 Dev.5,Abs Gap.3 Dev.10,Abs Gap.3 Dev.20,Abs Gap.3 Dev.40,Abs Gap.5 Dev.1,Abs Gap.5 Dev.3,Abs Gap.5 Dev.5,Abs Gap.5 Dev.10,Abs Gap.5 Dev.20,Abs Gap.5 Dev.40,Abs Gap.10 Dev.1,Abs Gap.10 Dev.3,Abs Gap.10 Dev.5,Abs Gap.10 Dev.10,Abs Gap.10 Dev.20,Abs Gap.10 Dev.40,Abs Gap.20 Dev.1,Abs Gap.20 Dev.3,Abs Gap.20 Dev.5,Abs Gap.20 Dev.10,Abs Gap.20 Dev.20,Abs Gap.20 Dev.40,Abs Gap.40 Dev.1,Abs Gap.40 Dev.3,Abs Gap.40 Dev.5,Abs Gap.40 Dev.10,Abs Gap.40 Dev.20,Abs Gap.40 Dev.40,Abs Gap Lvl,Abs Gap.1 Lvl.1,Abs Gap.1 Lvl.3,Abs Gap.1 Lvl.5,Abs Gap.1 Lvl.10,Abs Gap.1 Lvl.20,Abs Gap.1 Lvl.40,Abs Gap.3 Lvl.1,Abs Gap.3 Lvl.3,Abs Gap.3 Lvl.5,Abs Gap.3 Lvl.10,Abs Gap.3 Lvl.20,Abs Gap.3 Lvl.40,Abs Gap.5 Lvl.1,Abs Gap.5 Lvl.3,Abs Gap.5 Lvl.5,Abs Gap.5 Lvl.10,Abs Gap.5 Lvl.20,Abs Gap.5 Lvl.40,Abs Gap.10 Lvl.1,Abs Gap.10 Lvl.3,Abs Gap.10 Lvl.5,Abs Gap.10 Lvl.10,Abs Gap.10 Lvl.20,Abs Gap.10 Lvl.40,Abs Gap.20 Lvl.1,Abs Gap.20 Lvl.3,Abs Gap.20 Lvl.5,Abs Gap.20 Lvl.10,Abs Gap.20 Lvl.20,Abs Gap.20 Lvl.40,Abs Gap.40 Lvl.1,Abs Gap.40 Lvl.3,Abs Gap.40 Lvl.5,Abs Gap.40 Lvl.10,Abs Gap.40 Lvl.20,Abs Gap.40 Lvl.40
0,NaN,5.3,4.9,15.7,21.7,88.6,NaN,NaN,13.5,2.1,4.0,1.3,13.3,3.1,6.3,13.8,8.7,3.5,38.9,7.8,18.6,14.1,3.9,1.0,55.5,10.0,14.0,12.3,2.1,44.8,78.7,8.3,94.2,96.2,96.1,95.5,77.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,16.2,18.3,22.4,38.7,56.2,15.3,24.2,22.8,25.3,99.2,171.5,10.5,10.3,25.7,37.2,161.3,343.9,7.1,15.7,23.3,117.4,304.6,1212.1,33.4,47.4,50.9,78.7,673.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.296,0.501,0.081,0.333,0.270,0.191,0.782,0.453,0.199,0.375,0.642,0.126,0.351,0.192,0.573,0.571,0.665,0.179,0.953,0.551,0.838,1.018,1.247,0.110,2.345,2.832,2.607,1.601,0.341,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,8.9,18.0,21.0,7.4,91.9,NaN,NaN,15.5,2.5,4.1,1.2,13.6,3.1,7.8,14.2,8.6,3.5,39.2,7.8,18.9,13.9,3.1,0.5,55.7,10.1,12.9,12.1,0.9,46.0,78.7,8.4,94.7,96.4,96.1,95.5,77.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.6,16.2,18.3,22.5,39.5,56.3,14.9,24.1,22.9,25.4,99.5,171.8,10.4,10.3,26.4,38.5,161.1,344.7,6.9,15.7,24.4,120.3,305.7,1216.9,24.7,40.4,47.8,80.0,676.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.861,0.785,0.306,0.518,0.159,0.277,1.970,0.210,0.526,1.029,0.433,0.274,0.265,0.878,0.872,0.739,0.603,0.222,0.748,0.294,0.286,0.871,1.155,0.062,1.922,2.390,1.871,1.000,0.509,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0.4,6.5,24.7,4.2,93.7,NaN,NaN,19.1,3.3,3.7,1.2,13.9,3.1,10.2,13.4,8.1,3.6,39.4,7.9,17.1,13.2,2.2,0.0,55.8,10.2,13.0,12.7,0.0,47.2,78.6,8.4,94.9,96.5,96.2,95.4,76.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.4,16.3,18.5,22.5,39.9,56.4,11.5,24.8,23.0,25.3,99.7,172.1,12.7,10.9,26.7,39.5,160.9,345.5,6.9,15.8,25.4,124.1,308.2,1221.9,20.5,36.4,46.2,82.8,679.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.582,0.189,0.246,0.039,0.458,0.059,1.510,0.346,0.083,0.455,0.627,0.143,0.804,1.374,1.105,0.852,0.556,0.255,1.215,0.535,0.182,0.842,1.125,0.050,1.133,1.894,1.331,0.521,0.633,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,4.7,7.6,30.2,6.8,94.4,NaN,NaN,20.0,3.6,3.4,1.2,14.1,3.1,11.1,13.3,7.7,3.7,39.6,7.9,15.7,12.5,1.2,0.3,55.8,10.3,12.4,13.0,0.7,48.4,78.5,8.4,95.2,96.5,96.2,95.4,76.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.2,16.4,18.9,22.5,40.0,56.4,10.5,24.9,23.4,25.2,99.6,172.4,12.6,10.8,26.9,40.1,160

In [11]:
def CALCULATE_GAP_VARIATION(Src, IAbs, K, PRDS, SLGS):
    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS

    Df = pd.concat([ Src.loc[:, None:'Date'], IAbs.loc[:, None:f'Abs Gap {S6}'] ], axis=1)

    def add_variat_2nd(Val, G):
        Df[f'Gap.{G} Var.{P1}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P1*K, stp=(-1), rnd=2)
        Df[f'Gap.{G} Var.{P2}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P2*K, stp=(-1), rnd=2)
        Df[f'Gap.{G} Var.{P3}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P3*K, stp=(-1), rnd=2)
        Df[f'Gap.{G} Var.{P4}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P4*K, stp=(-1), rnd=2)
        Df[f'Gap.{G} Var.{P5}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P5*K, stp=(-1), rnd=2)
        Df[f'Gap.{G} Var.{P6}'] = _.Variat_2nd(Df, Val, By='Ticker.XY', N=P6*K, stp=(-1), rnd=2)
        pass

    Df['Gap Var'] = ''
    add_variat_2nd(f'Abs Gap {S1}', P1)
    add_variat_2nd(f'Abs Gap {S2}', P2)
    add_variat_2nd(f'Abs Gap {S3}', P3)
    add_variat_2nd(f'Abs Gap {S4}', P4)
    add_variat_2nd(f'Abs Gap {S5}', P5)
    add_variat_2nd(f'Abs Gap {S6}', P6)
    return Df


GVar = CALCULATE_GAP_VARIATION(Src2, IAbs, GRAN, PRDS, SLGS)
GVar.head()

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Abs Gap,Abs Gap 1Y,Abs Gap 3Y,Abs Gap 5Y,Abs Gap 10Y,Abs Gap 20Y,Abs Gap 40Y,Gap Var,Gap.1 Var.1,Gap.1 Var.3,Gap.1 Var.5,Gap.1 Var.10,Gap.1 Var.20,Gap.1 Var.40,Gap.3 Var.1,Gap.3 Var.3,Gap.3 Var.5,Gap.3 Var.10,Gap.3 Var.20,Gap.3 Var.40,Gap.5 Var.1,Gap.5 Var.3,Gap.5 Var.5,Gap.5 Var.10,Gap.5 Var.20,Gap.5 Var.40,Gap.10 Var.1,Gap.10 Var.3,Gap.10 Var.5,Gap.10 Var.10,Gap.10 Var.20,Gap.10 Var.40,Gap.20 Var.1,Gap.20 Var.3,Gap.20 Var.5,Gap.20 Var.10,Gap.20 Var.20,Gap.20 Var.40,Gap.40 Var.1,Gap.40 Var.3,Gap.40 Var.5,Gap.40 Var.10,Gap.40 Var.20,Gap.40 Var.40
0,0,AAPL,MSFT,AAPL-MSFT,2024-11-01,NaN,5.3,4.9,15.7,21.7,88.6,NaN,,-9.38,-5.56,-3.22,-8.99,-37.10,NaN,-6.34,-11.92,0.00,-5.58,-39.43,NaN,-2.61,8.03,-19.88,-26.63,-26.07,NaN,12.37,6.85,-12.45,-36.15,-35.61,NaN,-3.82,-4.17,-2.83,3.46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-1,AAPL,MSFT,AAPL-MSFT,2024-10-01,NaN,8.9,18.0,21.0,7.4,91.9,NaN,,-17.06,-8.79,-7.79,1.02,-28.78,NaN,9.87,3.60,7.08,15.46,-29.51,NaN,18.05,13.83,-10.96,-21.58,-21.02,NaN,-1.29,-16.22,-22.12,-44.18,-63.22,NaN,-1.99,-2.49,-1.69,7.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2,AAPL,MSFT,AAPL-MSFT,2024-09-01,NaN,0.4,6.5,24.7,4.2,93.7,NaN,,-8.56,-8.73,-18.04,-4.65,-31.42,NaN,3.90,5.13,-13.41,2.01,-31.82,NaN,15.46,23.59,-9.83,-16.14,-16.59,NaN,2.46,-10.40,-19.29,-46.29,-69.51,NaN,-1.53,-1.73,-1.07,7.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-3,AAPL,MSFT,AAPL-MSFT,2024-08-01,NaN,4.7,7.6,30.2,6.8,94.4,NaN,,-0.57,-8.24,-21.87,-2.33,-20.74,NaN,6.85,6.32,-12.73,2.87,-27.35,NaN,17.62,21.80,-14.90,-14.90,-9.39,NaN,1.42,-3.09,-16.63,-45.17,-72.66,NaN,-1.02,-1.27,-0.56,8.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-4,AAPL,MSFT,AAPL-MSFT,2024-07-01,NaN,10.4,2.7,25.5,8.4,94.8,NaN,,9.63,8.77,-3.92,2.32,-14.62,NaN,-7.14,-8.30,-13.62,-1.53,-31.30,NaN,2.45,15.14,-13.75,-17.05,-11.81,NaN,-1.09,1.59,-16.03,-44.47,-76.27,NaN,-0.97,-1.27,-0.61,9.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def COMBINE_FRAMES(Src, LS, PSts, Vars, VSts, IAbs, GVar):
    return pd.concat([
        Src, 
        PSts.loc[:, 'Cor':None], 
        LS, 
        Vars.loc[:, 'Vars.X+':None], 
        VSts.loc[:, 'Gap Avg':None], 
        IAbs, 
        GVar.loc[:, 'Gap Var':None]
    ], axis=1)

Frames = COMBINE_FRAMES(Src2, LS, PSts, Vars, VSts, IAbs, GVar)
Frames.head()

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,Price.X,Price.Y,LnPrc.X,LnPrc.Y,Cor,Cor 1Y,Cor 3Y,Cor 5Y,Cor 10Y,Cor 20Y,Cor 40Y,Long Short,Long 1Y,Short 1Y,Long 3Y,Short 3Y,Long 5Y,Short 5Y,Long 10Y,Short 10Y,Long 20Y,Short 20Y,Long 40Y,Short 40Y,LS Vars,Long Var +1Y,Short Var +1Y,Long Var +3Y,Short Var +3Y,Long Var +5Y,Short Var +5Y,Long Var +10Y,Short Var +10Y,Long Var +20Y,Short Var +20Y,Long Var +40Y,Short Var +40Y,LS Rtn,LS Rtn +1Y,LS Rtn +3Y,LS Rtn +5Y,LS Rtn +10Y,LS Rtn +20Y,LS Rtn +40Y,Vars.X+,Var.X +1Y,Var.X +3Y,Var.X +5Y,Var.X +10Y,Var.X +20Y,Var.X +40Y,Vars.Y+,Var.Y +1Y,Var.Y +3Y,Var.Y +5Y,Var.Y +10Y,Var.Y +20Y,Var.Y +40Y,Vars.X-,Var.X -1Y,Var.X -3Y,Var.X -5Y,Var.X -10Y,Var.X -20Y,Var.X -40Y,Vars.Y-,Var.Y -1Y,Var.Y -3Y,Var.Y -5Y,Var.Y -10Y,Var.Y -20Y,Var.Y -40Y,Gap,Gap 1Y,Gap 3Y,Gap 5Y,Gap 10Y,Gap 20Y,Gap 40Y,Gap Avg,Gap.1 Avg.1,Gap.1 Avg.3,Gap.1 Avg.5,Gap.1 Avg.10,Gap.1 Avg.20,Gap.1 Avg.40,Gap.3 Avg.1,Gap.3 Avg.3,Gap.3 Avg.5,Gap.3 Avg.10,Gap.3 Avg.20,Gap.3 Avg.40,Gap.5 Avg.1,Gap.5 Avg.3,Gap.5 Avg.5,Gap.5 Avg.10,Gap.5 Avg.20,Gap.5 Avg.40,Gap.10 Avg.1,Gap.10 Avg.3,Gap.10 Avg.5,Gap.10 Avg.10,Gap.10 Avg.20,Gap.10 Avg.40,Gap.20 Avg.1,Gap.20 Avg.3,Gap.20 Avg.5,Gap.20 Avg.10,Gap.20 Avg.20,Gap.20 Avg.40,Gap.40 Avg.1,Gap.40 Avg.3,Gap.40 Avg.5,Gap.40 Avg.10,Gap.40 Avg.20,Gap.40 Avg.40,Gap Dev,Gap.1 Dev.1,Gap.1 Dev.3,Gap.1 Dev.5,Gap.1 Dev.10,Gap.1 Dev.20,Gap.1 Dev.40,Gap.3 Dev.1,Gap.3 Dev.3,Gap.3 Dev.5,Gap.3 Dev.10,Gap.3 Dev.20,Gap.3 Dev.40,Gap.5 Dev.1,Gap.5 Dev.3,Gap.5 Dev.5,Gap.5 Dev.10,Gap.5 Dev.20,Gap.5 Dev.40,Gap.10 Dev.1,Gap.10 Dev.3,Gap.10 Dev.5,Gap.10 Dev.10,Gap.10 Dev.20,Gap.10 Dev.40,Gap.20 Dev.1,Gap.20 Dev.3,Gap.20 Dev.5,Gap.20 Dev.10,Gap.20 Dev.20,Gap.20 Dev.40,Gap.40 Dev.1,Gap.40 Dev.3,Gap.40 Dev.5,Gap.40 Dev.10,Gap.40 Dev.20,Gap.40 Dev.40,Gap Lvl,Gap.1 Lvl.1,Gap.1 Lvl.3,Gap.1 Lvl.5,Gap.1 Lvl.10,Gap.1 Lvl.20,Gap.1 Lvl.40,Gap.3 Lvl.1,Gap.3 Lvl.3,Gap.3 Lvl.5,Gap.3 Lvl.10,Gap.3 Lvl.20,Gap.3 Lvl.40,Gap.5 Lvl.1,Gap.5 Lvl.3,Gap.5 Lvl.5,Gap.5 Lvl.10,Gap.5 Lvl.20,Gap.5 Lvl.40,Gap.10 Lvl.1,Gap.10 Lvl.3,Gap.10 Lvl.5,Gap.10 Lvl.10,Gap.10 Lvl.20,Gap.10 Lvl.40,Gap.20 Lvl.1,Gap.20 Lvl.3,Gap.20 Lvl.5,Gap.20 Lvl.10,Gap.20 Lvl.20,Gap.20 Lvl.40,Gap.40 Lvl.1,Gap.40 Lvl.3,Gap.40 Lvl.5,Gap.40 Lvl.10,Gap.40 Lvl.20,Gap.40 Lvl.40,Abs Gap,Abs Gap 1Y,Abs Gap 3Y,Abs Gap 5Y,Abs Gap 10Y,Abs Gap 20Y,Abs Gap 40Y,Abs Gap Avg,Abs Gap.1 Avg.1,Abs Gap.1 Avg.3,Abs Gap.1 Avg.5,Abs Gap.1 Avg.10,Abs Gap.1 Avg.20,Abs Gap.1 Avg.40,Abs Gap.3 Avg.1,Abs Gap.3 Avg.3,Abs Gap.3 Avg.5,Abs Gap.3 Avg.10,Abs Gap.3 Avg.20,Abs Gap.3 Avg.40,Abs Gap.5 Avg.1,Abs Gap.5 Avg.3,Abs Gap.5 Avg.5,Abs Gap.5 Avg.10,Abs Gap.5 Avg.20,Abs Gap.5 Avg.40,Abs Gap.10 Avg.1,Abs Gap.10 Avg.3,Abs Gap.10 Avg.5,Abs Gap.10 Avg.10,Abs Gap.10 Avg.20,Abs Gap.10 Avg.40,Abs Gap.20 Avg.1,Abs Gap.20 Avg.3,Abs Gap.20 Avg.5,Abs Gap.20 Avg.10,Abs Gap.20 Avg.20,Abs Gap.20 Avg.40,Abs Gap.40 Avg.1,Abs Gap.40 Avg.3,Abs Gap.40 Avg.5,Abs Gap.40 Avg.10,Abs Gap.40 Avg.20,Abs Gap.40 Avg.40,Abs Gap Dev,Abs Gap.1 Dev.1,Abs Gap.1 Dev.3,Abs Gap.1 Dev.5,Abs Gap.1 Dev.10,Abs Gap.1 Dev.20,Abs Gap.1 Dev.40,Abs Gap.3 Dev.1,Abs Gap.3 Dev.3,Abs Gap.3 Dev.5,Abs Gap.3 Dev.10,Abs Gap.3 Dev.20,Abs Gap.3 Dev.40,Abs Gap.5 Dev.1,Abs Gap.5 Dev.3,Abs Gap.5 Dev.5,Abs Gap.5 Dev.10,Abs Gap.5 Dev.20,Abs Gap.5 Dev.40,Abs Gap.10 Dev.1,Abs Gap.10 Dev.3,Abs Gap.10 Dev.5,Abs Gap.10 Dev.10,Abs Gap.10 Dev.20,Abs Gap.10 Dev.40,Abs Gap.20 Dev.1,Abs Gap.20 Dev.3,Abs Gap.20 Dev.5,Abs Gap.20 Dev.10,Abs Gap.20 Dev.20,Abs Gap.20 Dev.40,Abs Gap.40 Dev.1,Abs Gap.40 Dev.3,Abs Gap.40 Dev.5,Abs Gap.40 Dev.10,Abs Gap.40 Dev.20,Abs Gap.40 Dev.40,Abs Gap Lvl,Abs Gap.1 Lvl.1,Abs Gap.1 Lvl.3,Abs Gap.1 Lvl.5,Abs Gap.1 Lvl.10,Abs Gap.1 Lvl.20,Abs Gap.1 Lvl.40,Abs Gap.3 Lvl.1,Abs Gap.3 Lvl.3,Abs Gap.3 Lvl.5,Abs Gap.3 Lvl.10,Abs Gap.3 Lvl.20,Abs Gap.3 Lvl.40,Abs Gap.5 Lvl.1,Abs Gap.5 Lvl.3,Abs Gap.5 Lvl.5,Abs Gap.5 Lvl.10,Abs Gap.5 Lvl.20,Abs Gap.5 Lvl.40,Abs Gap.10 Lvl.1,Abs Gap.10 Lvl.3,Abs Gap.10 Lvl.5,Abs Gap.10 Lvl.10,Abs Gap.10 Lvl.20,Abs Gap.10 Lvl.40,Abs Ga

In [13]:
def CALCULATE_RANKS(Frames, PRDS, SLGS):

    P1, P2, P3, P4, P5, P6 = PRDS
    S1, S2, S3, S4, S5, S6 = SLGS

    Rnk_Rtn = _.HRnk(Frames, _.pd_columns(Frames, f'LS Rtn +{S1}', f'LS Rtn +{S6}'), By='Date', prefix='High ', _as='float')
    Rnk_Rtn.insert(0, 'High LS Rtn Rank', '')

    Rnk_Cor = _.HRnk(Frames, _.pd_columns(Frames, f'Cor {S1}', f'Cor {S6}'), By='Date', prefix='High ', _as='float')
    Rnk_Cor.insert(0, 'High Cor Rank', '')
    
    Rnk_Inds = _.RANK('BANDED', Frames, feats=['Gap',None], Keys=['Ticker.X','Ticker.Y','Ticker.XY'], By='Date', method='dense')

    return Rnk_Rtn, Rnk_Cor, Rnk_Inds

    
Rnk_Rtn, Rnk_Cor, Rnk_Inds = CALCULATE_RANKS(Frames, PRDS, SLGS)

In [14]:
def COMBINE_RANKS(Src, Rnk_Rtn, Rnk_Cor, Rnk_Inds):
    return pd.concat([
        Src.loc[:, None:'Trans'], 
        Rnk_Rtn, 
        Rnk_Cor, 
        Rnk_Inds.loc[:, 'High Rank':None], 
    ], axis=1)


Ranks = COMBINE_RANKS(Src2, Rnk_Rtn, Rnk_Cor, Rnk_Inds)
Ranks

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,High LS Rtn Rank,High LS Rtn +1Y,High LS Rtn +3Y,High LS Rtn +5Y,High LS Rtn +10Y,High LS Rtn +20Y,High LS Rtn +40Y,High Cor Rank,High Cor 1Y,High Cor 3Y,High Cor 5Y,High Cor 10Y,High Cor 20Y,High Cor 40Y,High Rank,Low Rank,High Gap,Low Gap,High Gap 1Y,Low Gap 1Y,High Gap 3Y,Low Gap 3Y,High Gap 5Y,Low Gap 5Y,High Gap 10Y,Low Gap 10Y,High Gap 20Y,Low Gap 20Y,High Gap 40Y,Low Gap 40Y,High Gap Avg,Low Gap Avg,High Gap.1 Avg.1,Low Gap.1 Avg.1,High Gap.1 Avg.3,Low Gap.1 Avg.3,High Gap.1 Avg.5,Low Gap.1 Avg.5,High Gap.1 Avg.10,Low Gap.1 Avg.10,High Gap.1 Avg.20,Low Gap.1 Avg.20,High Gap.1 Avg.40,Low Gap.1 Avg.40,High Gap.3 Avg.1,Low Gap.3 Avg.1,High Gap.3 Avg.3,Low Gap.3 Avg.3,High Gap.3 Avg.5,Low Gap.3 Avg.5,High Gap.3 Avg.10,Low Gap.3 Avg.10,High Gap.3 Avg.20,Low Gap.3 Avg.20,High Gap.3 Avg.40,Low Gap.3 Avg.40,High Gap.5 Avg.1,Low Gap.5 Avg.1,High Gap.5 Avg.3,Low Gap.5 Avg.3,High Gap.5 Avg.5,Low Gap.5 Avg.5,High Gap.5 Avg.10,Low Gap.5 Avg.10,High Gap.5 Avg.20,Low Gap.5 Avg.20,High Gap.5 Avg.40,Low Gap.5 Avg.40,High Gap.10 Avg.1,Low Gap.10 Avg.1,High Gap.10 Avg.3,Low Gap.10 Avg.3,High Gap.10 Avg.5,Low Gap.10 Avg.5,High Gap.10 Avg.10,Low Gap.10 Avg.10,High Gap.10 Avg.20,Low Gap.10 Avg.20,High Gap.10 Avg.40,Low Gap.10 Avg.40,High Gap.20 Avg.1,Low Gap.20 Avg.1,High Gap.20 Avg.3,Low Gap.20 Avg.3,High Gap.20 Avg.5,Low Gap.20 Avg.5,High Gap.20 Avg.10,Low Gap.20 Avg.10,High Gap.20 Avg.20,Low Gap.20 Avg.20,High Gap.20 Avg.40,Low Gap.20 Avg.40,High Gap.40 Avg.1,Low Gap.40 Avg.1,High Gap.40 Avg.3,Low Gap.40 Avg.3,High Gap.40 Avg.5,Low Gap.40 Avg.5,High Gap.40 Avg.10,Low Gap.40 Avg.10,High Gap.40 Avg.20,Low Gap.40 Avg.20,High Gap.40 Avg.40,Low Gap.40 Avg.40,High Gap Dev,Low Gap Dev,High Gap.1 Dev.1,Low Gap.1 Dev.1,High Gap.1 Dev.3,Low Gap.1 Dev.3,High Gap.1 Dev.5,Low Gap.1 Dev.5,High Gap.1 Dev.10,Low Gap.1 Dev.10,High Gap.1 Dev.20,Low Gap.1 Dev.20,High Gap.1 Dev.40,Low Gap.1 Dev.40,High Gap.3 Dev.1,Low Gap.3 Dev.1,High Gap.3 Dev.3,Low Gap.3 Dev.3,High Gap.3 Dev.5,Low Gap.3 Dev.5,High Gap.3 Dev.10,Low Gap.3 Dev.10,High Gap.3 Dev.20,Low Gap.3 Dev.20,High Gap.3 Dev.40,Low Gap.3 Dev.40,High Gap.5 Dev.1,Low Gap.5 Dev.1,High Gap.5 Dev.3,Low Gap.5 Dev.3,High Gap.5 Dev.5,Low Gap.5 Dev.5,High Gap.5 Dev.10,Low Gap.5 Dev.10,High Gap.5 Dev.20,Low Gap.5 Dev.20,High Gap.5 Dev.40,Low Gap.5 Dev.40,High Gap.10 Dev.1,Low Gap.10 Dev.1,High Gap.10 Dev.3,Low Gap.10 Dev.3,High Gap.10 Dev.5,Low Gap.10 Dev.5,High Gap.10 Dev.10,Low Gap.10 Dev.10,High Gap.10 Dev.20,Low Gap.10 Dev.20,High Gap.10 Dev.40,Low Gap.10 Dev.40,High Gap.20 Dev.1,Low Gap.20 Dev.1,High Gap.20 Dev.3,Low Gap.20 Dev.3,High Gap.20 Dev.5,Low Gap.20 Dev.5,High Gap.20 Dev.10,Low Gap.20 Dev.10,High Gap.20 Dev.20,Low Gap.20 Dev.20,High Gap.20 Dev.40,Low Gap.20 Dev.40,High Gap.40 Dev.1,Low Gap.40 Dev.1,High Gap.40 Dev.3,Low Gap.40 Dev.3,High Gap.40 Dev.5,Low Gap.40 Dev.5,High Gap.40 Dev.10,Low Gap.40 Dev.10,High Gap.40 Dev.20,Low Gap.40 Dev.20,High Gap.40 Dev.40,Low Gap.40 Dev.40,High Gap Lvl,Low Gap Lvl,High Gap.1 Lvl.1,Low Gap.1 Lvl.1,High Gap.1 Lvl.3,Low Gap.1 Lvl.3,High Gap.1 Lvl.5,Low Gap.1 Lvl.5,High Gap.1 Lvl.10,Low Gap.1 Lvl.10,High Gap.1 Lvl.20,Low Gap.1 Lvl.20,High Gap.1 Lvl.40,Low Gap.1 Lvl.40,High Gap.3 Lvl.1,Low Gap.3 Lvl.1,High Gap.3 Lvl.3,Low Gap.3 Lvl.3,High Gap.3 Lvl.5,Low Gap.3 Lvl.5,High Gap.3 Lvl.10,Low Gap.3 Lvl.10,High Gap.3 Lvl.20,Low Gap.3 Lvl.20,High Gap.3 Lvl.40,Low Gap.3 Lvl.40,High Gap.5 Lvl.1,Low Gap.5 Lvl.1,High Gap.5 Lvl.3,Low Gap.5 Lvl.3,High Gap.5 Lvl.5,Low Gap.5 Lvl.5,High Gap.5 Lvl.10,Low Gap.5 Lvl.10,High Gap.5 Lvl.20,Low Gap.5 Lvl.20,High Gap.5 Lvl.40,Low Gap.5 Lvl.40,High Gap.10 Lvl.1,Low Gap.10 Lvl.1,High Gap.10 Lvl.3,Low Gap.10 Lvl.3,High Gap.10 Lvl.5,Low Gap.10 Lvl.5,High Gap.10 Lvl.10,Low Gap.10 Lvl.10,High Gap.10 Lvl.20,Low Gap.10 Lvl.20,High Gap.10 Lvl.40,Low Gap.10 Lvl.40,High Gap.20 Lvl.1,Low Gap.20 Lvl.1,High Gap.20 Lvl.3,Low Gap.20 Lvl.3,High Gap.20 Lvl.5,Low Gap.20 Lvl.5,High Gap.20 Lvl.10,Low Gap.20 Lvl.10,High Gap

In [15]:
def COMBINE_DATAS(Frames, Ranks):
    return pd.concat([
        Frames,
        Ranks.loc[:, 'High LS Rtn Rank':None], 
    ], axis=1)

Data = COMBINE_DATAS(Frames, Ranks)
Data

N Ticker.X Ticker.Y  Ticker.XY       Date  Trans     Price.X  \
0        0     AAPL     MSFT  AAPL-MSFT 2024-11-01      0  224.229996   
1       -1     AAPL     MSFT  AAPL-MSFT 2024-10-01      0  225.661728   
2       -2     AAPL     MSFT  AAPL-MSFT 2024-09-01      0  232.743942   
3       -3     AAPL     MSFT  AAPL-MSFT 2024-08-01      0  228.483871   
4       -4     AAPL     MSFT  AAPL-MSFT 2024-07-01      0  221.579468   
...    ...      ...      ...        ...        ...    ...         ...   
48241 -341     EPAM     MKTX  EPAM-MKTX 2012-07-01      0   16.040001   
48242 -342     EPAM     MKTX  EPAM-MKTX 2012-06-01      0   16.990000   
48243 -343     EPAM     MKTX  EPAM-MKTX 2012-05-01      0   17.500000   
48244 -344     EPAM     MKTX  EPAM-MKTX 2012-04-01      0   20.639999   
48245 -345     EPAM     MKTX  EPAM-MKTX 2012-03-01      0   20.520000   

          Price.Y   LnPrc.X   LnPrc.Y Cor  Cor 1Y  Cor 3Y  Cor 5Y  Cor 10Y  \
0      423.029999  5.412672  6.047443        48.2    88.6    93.9     98.1   
1      406.350006  5.419037  6.007215        46.1    87.8    94.1     98.1   
2      430.299988  5.449939  6.064483        58.2    86.1    94.4     98.1   
3      416.389496  5.431466  6.031621        57.8    85.1    94.6     98.0   
4      417.597321  5.400781  6.034518        50.3    84.3    94.6     98.0   
...           ...       ...       ...  ..     ...     ...     ...      ...   
48241   27.098576  2.775086  3.299481         NaN     NaN     NaN      NaN   
48242   23.888359  2.832625  3.173391         NaN     NaN     NaN      NaN   
48243   28.857672  2.862201  3.362376         NaN     NaN     NaN      NaN   
48244   30.662949  3.027231  3.423055         NaN     NaN     NaN      NaN   
48245   33.326183  3.021400  3.506343         NaN     NaN     NaN      NaN   

       Cor 20Y  Cor 40Y Long Short Long 1Y Short 1Y Long 3Y Short 3Y Long 5Y  \
0         93.1     80.3        NaN    MSFT     AAPL    MSFT     AAPL    MSFT   
1         92.9     80.2        NaN    MSFT     AAPL    MSFT     AAPL    MSFT   
2         92.7     80.1        NaN    AAPL     MSFT    MSFT     AAPL    MSFT   
3         92.3     79.9        NaN    AAPL     MSFT    MSFT     AAPL    MSFT   
4         92.0     79.8        NaN    AAPL     MSFT    MSFT     AAPL    MSFT   
...        ...      ...        ...     ...      ...     ...      ...     ...   
48241      NaN      NaN        NaN                                             
48242      NaN      NaN        NaN                                             
48243      NaN      NaN        NaN                                             
48244      NaN      NaN        NaN                                             
48245      NaN      NaN        NaN                                             

      Short 5Y Long 10Y Short 10Y Long 20Y Short 20Y Long 40Y Short 40Y  \
0         AAPL     AAPL      MSFT     MSFT      AAPL                      
1         AAPL     AAPL      MSFT     MSFT      AAPL                      
2         AAPL     AAPL      MSFT     MSFT      AAPL                      
3         AAPL     AAPL      MSFT     MSFT      AAPL                      
4         AAPL     AAPL      MSFT     MSFT      AAPL                      
...        ...      ...       ...      ...       ...      ...       ...   
48241                                                                     
48242                                                                     
48243                                                                     
48244                                                                     
48245                                                                     

      LS Vars  Long Var +1Y  Short Var +1Y  Long Var +3Y  Short Var +3Y  \
0                       NaN            NaN           NaN            NaN   
1                       NaN            NaN           NaN            NaN   
2                       NaN            NaN           NaN            NaN   
3                       NaN      

In [16]:
def SORT_SCREENER(Data):
    Scrn = Data.sort_values('Date', ascending=False) . reset_index(drop=1)
    Scrn['N'] = _.Nth(Scrn, 'Date', By='', sign=(-1), asc=0, adj=(+1))
    return Scrn

Scrn = SORT_SCREENER(Data)
Scrn

N Ticker.X Ticker.Y  Ticker.XY       Date  Trans     Price.X  \
0        0     AAPL     MSFT  AAPL-MSFT 2024-11-01      0  224.229996   
1        0     IDXX     ANET  IDXX-ANET 2024-11-01      0  426.109985   
2        0     CPRT      WST   CPRT-WST 2024-11-01      0   57.180000   
3        0     CPRT     MPWR  CPRT-MPWR 2024-11-01      0   57.180000   
4        0     CPRT     POOL  CPRT-POOL 2024-11-01      0   57.180000   
...    ...      ...      ...        ...        ...    ...         ...   
48241 -478     AAPL      SHW   AAPL-SHW 1985-01-01      1    0.099694   
48242 -478     AAPL      WST   AAPL-WST 1985-01-01      1    0.099694   
48243 -478     AAPL      UNH   AAPL-UNH 1985-01-01      1    0.099694   
48244 -478      SHW      WST    SHW-WST 1985-01-01      1    0.736926   
48245 -478      UNH      WST    UNH-WST 1985-01-01      1    0.155194   

          Price.Y   LnPrc.X   LnPrc.Y Cor  Cor 1Y  Cor 3Y  Cor 5Y  Cor 10Y  \
0      423.029999  5.412672  6.047443        48.2    88.6    93.9     98.1   
1      395.549988  6.054697  5.980277       -73.7    34.8    40.8     88.0   
2      327.679993  4.046204  5.792038        10.2    37.2    65.7     95.8   
3      644.099976  4.046204  6.467854        10.1    87.1    92.1     97.6   
4      356.480011  4.046204  5.876278       -11.6     5.0    52.9     96.5   
...           ...       ...       ...  ..     ...     ...     ...      ...   
48241    0.736926 -2.305651 -0.305268         NaN     NaN     NaN      NaN   
48242    1.445494 -2.305651  0.368451         NaN     NaN     NaN      NaN   
48243    0.155194 -2.305651 -1.863078         NaN     NaN     NaN      NaN   
48244    1.445494 -0.305268  0.368451         NaN     NaN     NaN      NaN   
48245    1.445494 -1.863078  0.368451         NaN     NaN     NaN      NaN   

       Cor 20Y  Cor 40Y Long Short Long 1Y Short 1Y Long 3Y Short 3Y Long 5Y  \
0         93.1     80.3        NaN    MSFT     AAPL    MSFT     AAPL    MSFT   
1         88.7      NaN        NaN    IDXX     ANET    IDXX     ANET    IDXX   
2         98.2     96.1        NaN     WST     CPRT     WST     CPRT     WST   
3         98.3     98.3        NaN    CPRT     MPWR    MPWR     CPRT    CPRT   
4         96.4     97.5        NaN    POOL     CPRT    POOL     CPRT    POOL   
...        ...      ...        ...     ...      ...     ...      ...     ...   
48241      NaN      NaN        NaN                                             
48242      NaN      NaN        NaN                                             
48243      NaN      NaN        NaN                                             
48244      NaN      NaN        NaN                                             
48245      NaN      NaN        NaN                                             

      Short 5Y Long 10Y Short 10Y Long 20Y Short 20Y Long 40Y Short 40Y  \
0         AAPL     AAPL      MSFT     MSFT      AAPL                      
1         ANET     IDXX      ANET                                         
2         CPRT      WST      CPRT      WST      CPRT                      
3         MPWR     CPRT      MPWR                                         
4         CPRT     POOL      CPRT     POOL      CPRT                      
...        ...      ...       ...      ...       ...      ...       ...   
48241                                                                     
48242                                                                     
48243                                                                     
48244                                                                     
48245                                                                     

      LS Vars  Long Var +1Y  Short Var +1Y  Long Var +3Y  Short Var +3Y  \
0                       NaN            NaN           NaN            NaN   
1                       NaN            NaN           NaN            NaN   
2                       NaN            NaN           NaN            NaN   
3                       NaN      

# Section 3

In [17]:
def SELECT(Scrn, GRAN):
    Df = Scrn[(Scrn['N']<=(-GRAN)) & (Scrn['Trans']==1)] . reset_index(drop=1)
    Df['N'] = _.Nth(Df, 'Date', By='', sign=(-1), asc=0, adj=0)
    return Df

Slc = SELECT(Scrn, GRAN)
Slc

N Ticker.X Ticker.Y   Ticker.XY       Date  Trans     Price.X  \
0     -1     MSFT    GOOGL  MSFT-GOOGL 2023-01-01      1  244.296844   
1     -1      UNH      RMD     UNH-RMD 2023-01-01      1  486.062103   
2     -1     AAPL        V      AAPL-V 2023-01-01      1  142.788406   
3     -1     ODFL     POOL   ODFL-POOL 2023-01-01      1  165.231613   
4     -1        V     EPAM      V-EPAM 2023-01-01      1  227.094696   
...   ..      ...      ...         ...        ...    ...         ...   
3748 -39     AAPL      SHW    AAPL-SHW 1985-01-01      1    0.099694   
3749 -39     AAPL      WST    AAPL-WST 1985-01-01      1    0.099694   
3750 -39     AAPL      UNH    AAPL-UNH 1985-01-01      1    0.099694   
3751 -39      SHW      WST     SHW-WST 1985-01-01      1    0.736926   
3752 -39      UNH      WST     UNH-WST 1985-01-01      1    0.155194   

         Price.Y   LnPrc.X   LnPrc.Y Cor  Cor 1Y  Cor 3Y  Cor 5Y  Cor 10Y  \
0      98.595856  5.498384  4.591029        96.1    96.3    94.9     98.2   
1     224.076477  6.186336  5.411987         0.8    69.8    86.7     95.9   
2     227.094696  4.961364  5.425367        41.3    69.3    89.7     94.8   
3     377.332397  5.107348  5.933126        40.4    83.7    96.0     97.8   
4     332.649994  5.425367  5.807091       -29.2    59.5    89.0     96.9   
...          ...       ...       ...  ..     ...     ...     ...      ...   
3748    0.736926 -2.305651 -0.305268         NaN     NaN     NaN      NaN   
3749    1.445494 -2.305651  0.368451         NaN     NaN     NaN      NaN   
3750    0.155194 -2.305651 -1.863078         NaN     NaN     NaN      NaN   
3751    1.445494 -0.305268  0.368451         NaN     NaN     NaN      NaN   
3752    1.445494 -1.863078  0.368451         NaN     NaN     NaN      NaN   

      Cor 20Y  Cor 40Y Long Short Long 1Y Short 1Y Long 3Y Short 3Y Long 5Y  \
0        95.6      NaN        NaN   GOOGL     MSFT   GOOGL     MSFT   GOOGL   
1        95.6     95.7        NaN     RMD      UNH     RMD      UNH     UNH   
2        95.9      NaN        NaN    AAPL        V       V     AAPL       V   
3        96.6     95.6        NaN    POOL     ODFL    POOL     ODFL    POOL   
4        97.6      NaN        NaN    EPAM        V       V     EPAM       V   
...       ...      ...        ...     ...      ...     ...      ...     ...   
3748      NaN      NaN        NaN                                             
3749      NaN      NaN        NaN                                             
3750      NaN      NaN        NaN                                             
3751      NaN      NaN        NaN                                             
3752      NaN      NaN        NaN                                             

     Short 5Y Long 10Y Short 10Y Long 20Y Short 20Y Long 40Y Short 40Y  \
0        MSFT    GOOGL      MSFT                                         
1         RMD      RMD       UNH      UNH       RMD                      
2        AAPL        V      AAPL                                         
3        ODFL     POOL      ODFL     POOL      ODFL                      
4        EPAM        V      EPAM                                         
...       ...      ...       ...      ...       ...      ...       ...   
3748                                                                     
3749                                                                     
3750                                                                     
3751                                                                     
3752                                                                     

     LS Vars  Long Var +1Y  Short Var +1Y  Long Var +3Y  Short Var +3Y  \
0                    41.74          61.86           NaN            NaN   
1                   -15.90           4.06           NaN            NaN   
2                    28.51          19.64           NaN            NaN   
3                    -2.53          17.85           NaN            NaN   
4        

In [22]:
def RANK_COMBINATION(Src,  Keys, By,  Ind_1, Ind_2,  WGHTS, NORM_WGHTS=False):

    WGHTS = _.TotalNorm(WGHTS, norm=NORM_WGHTS)
    Keys  = np.ravel(Keys).tolist()
    By    = np.ravel(By).tolist()
    Df    = Src[[*Keys,*By]].copy()

    Df['Rank Comb'] = ''
    Df['Art Comb']  = (Src[Ind_1] *  WGHTS[0]  +  Src[Ind_2] *  WGHTS[1])
    Df['Pwr Comb']  = (Src[Ind_1] ** WGHTS[0]  +  Src[Ind_2] ** WGHTS[1])
    Df['Geo Comb']  = (Src[Ind_1] ** WGHTS[0]  *  Src[Ind_2] ** WGHTS[1])

    Df['High Art']  = Df.groupby(By)['Art Comb'].rank(method='dense', ascending=1)
    Df['Low Art']   = Df.groupby(By)['Art Comb'].rank(method='dense', ascending=0)

    Df['High Pwr']  = Df.groupby(By)['Pwr Comb'].rank(method='dense', ascending=1)
    Df['Low Pwr']   = Df.groupby(By)['Pwr Comb'].rank(method='dense', ascending=0)

    Df['High Geo']  = Df.groupby(By)['Geo Comb'].rank(method='dense', ascending=1)
    Df['Low Geo']   = Df.groupby(By)['Geo Comb'].rank(method='dense', ascending=0)
    return Df


RnkComb = RANK_COMBINATION(Slc,  ['N','Ticker.XY'],['Date'],  RNK_1, RNK_2,  WGHTS, NORM_WGHTS)
RnkComb.head()

,N,Ticker.XY,Date,Rank Comb,Art Comb,Pwr Comb,Geo Comb,High Art,Low Art,High Pwr,Low Pwr,High Geo,Low Geo
0,-1,MSFT-GOOGL,2023-01-01,,57.5,12.044359,15.033296,28.0,103.0,18.0,172.0,6.0,183.0
1,-1,UNH-RMD,2023-01-01,,123.0,22.121293,121.675799,95.0,36.0,152.0,38.0,153.0,36.0
2,-1,AAPL-V,2023-01-01,,76.0,17.211103,72.111026,55.0,76.0,95.0,95.0,100.0,89.0
3,-1,ODFL-POOL,2023-01-01,,121.0,21.924218,119.335661,93.0,38.0,147.0,43.0,147.0,42.0
4,-1,V-EPAM,2023-01-01,,153.5,24.777413,153.460093,124.0,7.0,183.0,7.0,182.0,7.0


In [24]:
def TOP_CLASSIFCATION(RnkComb, RankBy, TOP):
    return _.pd_isin_top(RnkComb, Keys=['N','Ticker.XY','Date'], Rank=RankBy, TOP=TOP,  with_RnkVal=1)

InTop = TOP_CLASSIFCATION(RnkComb, RankBy=RCOMB, TOP=TOP)
InTop

,N,Ticker.XY,Date,Rnk Val,Isin Top,InTop 5,InTop 10,InTop 25,InTop 50,InTop 100,InTop 250,InTop 500
0,-1,MSFT-GOOGL,2023-01-01,28.0,,0,0,0,1,1,1,1
1,-1,UNH-RMD,2023-01-01,95.0,,0,0,0,0,1,1,1
2,-1,AAPL-V,2023-01-01,55.0,,0,0,0,0,1,1,1
3,-1,ODFL-POOL,2023-01-01,93.0,,0,0,0,0,1,1,1
4,-1,V-EPAM,2023-01-01,124.0,,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
3748,-39,AAPL-SHW,1985-01-01,NaN,,0,0,0,0,0,0,0
3749,-39,AAPL-WST,1985-01-01,NaN,,0,0,0,0,0,0,0
3750,-39,AAPL-UNH,1985-01-01,NaN,,0,0,0,0,0,0,0
3751,-39,SHW-WST,1985-01-01,NaN,,0,0,0,0,0,0,0


In [25]:
def COMBINE_CLASS_RCOMB(InTop, RnkComb):
    return pd.concat([  InTop, RnkComb.loc[:,'Rank Comb':None]  ], axis=1)

Class_RComb = COMBINE_CLASS_RCOMB(InTop, RnkComb)
Class_RComb

,N,Ticker.XY,Date,Rnk Val,Isin Top,InTop 5,InTop 10,InTop 25,InTop 50,InTop 100,InTop 250,InTop 500,Rank Comb,Art Comb,Pwr Comb,Geo Comb,High Art,Low Art,High Pwr,Low Pwr,High Geo,Low Geo
0,-1,MSFT-GOOGL,2023-01-01,28.0,,0,0,0,1,1,1,1,,57.5,12.044359,15.033296,28.0,103.0,18.0,172.0,6.0,183.0
1,-1,UNH-RMD,2023-01-01,95.0,,0,0,0,0,1,1,1,,123.0,22.121293,121.675799,95.0,36.0,152.0,38.0,153.0,36.0
2,-1,AAPL-V,2023-01-01,55.0,,0,0,0,0,1,1,1,,76.0,17.211103,72.111026,55.0,76.0,95.0,95.0,100.0,89.0
3,-1,ODFL-POOL,2023-01-01,93.0,,0,0,0,0,1,1,1,,121.0,21.924218,119.335661,93.0,38.0,147.0,43.0,147.0,42.0
4,-1,V-EPAM,2023-01-01,124.0,,0,0,0,0,0,1,1,,153.5,24.777413,153.460093,124.0,7.0,183.0,7.0,182.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3748,-39,AAPL-SHW,1985-01-01,NaN,,0,0,0,0,0,0,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3749,-39,AAPL-WST,1985-01-01,NaN,,0,0,0,0,0,0,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3750,-39,AAPL-UNH,1985-01-01,NaN,,0,0,0,0,0,0,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3751,-39,SHW-WST,1985-01-01,NaN,,0,0,0,0,0,0,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
def INSERT_CLASS_RCOMB_IN_DFRAME(Src, Class_RComb): 
    return _.pd_insert_df(Src, 'before','High LS Rtn Rank',  Class_RComb.loc[:, 'Rnk Val':None])

DFrame = INSERT_CLASS_RCOMB_IN_DFRAME(Slc, Class_RComb)
DFrame.head()

,N,Ticker.X,Ticker.Y,Ticker.XY,Date,Trans,Price.X,Price.Y,LnPrc.X,LnPrc.Y,Cor,Cor 1Y,Cor 3Y,Cor 5Y,Cor 10Y,Cor 20Y,Cor 40Y,Long Short,Long 1Y,Short 1Y,Long 3Y,Short 3Y,Long 5Y,Short 5Y,Long 10Y,Short 10Y,Long 20Y,Short 20Y,Long 40Y,Short 40Y,LS Vars,Long Var +1Y,Short Var +1Y,Long Var +3Y,Short Var +3Y,Long Var +5Y,Short Var +5Y,Long Var +10Y,Short Var +10Y,Long Var +20Y,Short Var +20Y,Long Var +40Y,Short Var +40Y,LS Rtn,LS Rtn +1Y,LS Rtn +3Y,LS Rtn +5Y,LS Rtn +10Y,LS Rtn +20Y,LS Rtn +40Y,Vars.X+,Var.X +1Y,Var.X +3Y,Var.X +5Y,Var.X +10Y,Var.X +20Y,Var.X +40Y,Vars.Y+,Var.Y +1Y,Var.Y +3Y,Var.Y +5Y,Var.Y +10Y,Var.Y +20Y,Var.Y +40Y,Vars.X-,Var.X -1Y,Var.X -3Y,Var.X -5Y,Var.X -10Y,Var.X -20Y,Var.X -40Y,Vars.Y-,Var.Y -1Y,Var.Y -3Y,Var.Y -5Y,Var.Y -10Y,Var.Y -20Y,Var.Y -40Y,Gap,Gap 1Y,Gap 3Y,Gap 5Y,Gap 10Y,Gap 20Y,Gap 40Y,Gap Avg,Gap.1 Avg.1,Gap.1 Avg.3,Gap.1 Avg.5,Gap.1 Avg.10,Gap.1 Avg.20,Gap.1 Avg.40,Gap.3 Avg.1,Gap.3 Avg.3,Gap.3 Avg.5,Gap.3 Avg.10,Gap.3 Avg.20,Gap.3 Avg.40,Gap.5 Avg.1,Gap.5 Avg.3,Gap.5 Avg.5,Gap.5 Avg.10,Gap.5 Avg.20,Gap.5 Avg.40,Gap.10 Avg.1,Gap.10 Avg.3,Gap.10 Avg.5,Gap.10 Avg.10,Gap.10 Avg.20,Gap.10 Avg.40,Gap.20 Avg.1,Gap.20 Avg.3,Gap.20 Avg.5,Gap.20 Avg.10,Gap.20 Avg.20,Gap.20 Avg.40,Gap.40 Avg.1,Gap.40 Avg.3,Gap.40 Avg.5,Gap.40 Avg.10,Gap.40 Avg.20,Gap.40 Avg.40,Gap Dev,Gap.1 Dev.1,Gap.1 Dev.3,Gap.1 Dev.5,Gap.1 Dev.10,Gap.1 Dev.20,Gap.1 Dev.40,Gap.3 Dev.1,Gap.3 Dev.3,Gap.3 Dev.5,Gap.3 Dev.10,Gap.3 Dev.20,Gap.3 Dev.40,Gap.5 Dev.1,Gap.5 Dev.3,Gap.5 Dev.5,Gap.5 Dev.10,Gap.5 Dev.20,Gap.5 Dev.40,Gap.10 Dev.1,Gap.10 Dev.3,Gap.10 Dev.5,Gap.10 Dev.10,Gap.10 Dev.20,Gap.10 Dev.40,Gap.20 Dev.1,Gap.20 Dev.3,Gap.20 Dev.5,Gap.20 Dev.10,Gap.20 Dev.20,Gap.20 Dev.40,Gap.40 Dev.1,Gap.40 Dev.3,Gap.40 Dev.5,Gap.40 Dev.10,Gap.40 Dev.20,Gap.40 Dev.40,Gap Lvl,Gap.1 Lvl.1,Gap.1 Lvl.3,Gap.1 Lvl.5,Gap.1 Lvl.10,Gap.1 Lvl.20,Gap.1 Lvl.40,Gap.3 Lvl.1,Gap.3 Lvl.3,Gap.3 Lvl.5,Gap.3 Lvl.10,Gap.3 Lvl.20,Gap.3 Lvl.40,Gap.5 Lvl.1,Gap.5 Lvl.3,Gap.5 Lvl.5,Gap.5 Lvl.10,Gap.5 Lvl.20,Gap.5 Lvl.40,Gap.10 Lvl.1,Gap.10 Lvl.3,Gap.10 Lvl.5,Gap.10 Lvl.10,Gap.10 Lvl.20,Gap.10 Lvl.40,Gap.20 Lvl.1,Gap.20 Lvl.3,Gap.20 Lvl.5,Gap.20 Lvl.10,Gap.20 Lvl.20,Gap.20 Lvl.40,Gap.40 Lvl.1,Gap.40 Lvl.3,Gap.40 Lvl.5,Gap.40 Lvl.10,Gap.40 Lvl.20,Gap.40 Lvl.40,Abs Gap,Abs Gap 1Y,Abs Gap 3Y,Abs Gap 5Y,Abs Gap 10Y,Abs Gap 20Y,Abs Gap 40Y,Abs Gap Avg,Abs Gap.1 Avg.1,Abs Gap.1 Avg.3,Abs Gap.1 Avg.5,Abs Gap.1 Avg.10,Abs Gap.1 Avg.20,Abs Gap.1 Avg.40,Abs Gap.3 Avg.1,Abs Gap.3 Avg.3,Abs Gap.3 Avg.5,Abs Gap.3 Avg.10,Abs Gap.3 Avg.20,Abs Gap.3 Avg.40,Abs Gap.5 Avg.1,Abs Gap.5 Avg.3,Abs Gap.5 Avg.5,Abs Gap.5 Avg.10,Abs Gap.5 Avg.20,Abs Gap.5 Avg.40,Abs Gap.10 Avg.1,Abs Gap.10 Avg.3,Abs Gap.10 Avg.5,Abs Gap.10 Avg.10,Abs Gap.10 Avg.20,Abs Gap.10 Avg.40,Abs Gap.20 Avg.1,Abs Gap.20 Avg.3,Abs Gap.20 Avg.5,Abs Gap.20 Avg.10,Abs Gap.20 Avg.20,Abs Gap.20 Avg.40,Abs Gap.40 Avg.1,Abs Gap.40 Avg.3,Abs Gap.40 Avg.5,Abs Gap.40 Avg.10,Abs Gap.40 Avg.20,Abs Gap.40 Avg.40,Abs Gap Dev,Abs Gap.1 Dev.1,Abs Gap.1 Dev.3,Abs Gap.1 Dev.5,Abs Gap.1 Dev.10,Abs Gap.1 Dev.20,Abs Gap.1 Dev.40,Abs Gap.3 Dev.1,Abs Gap.3 Dev.3,Abs Gap.3 Dev.5,Abs Gap.3 Dev.10,Abs Gap.3 Dev.20,Abs Gap.3 Dev.40,Abs Gap.5 Dev.1,Abs Gap.5 Dev.3,Abs Gap.5 Dev.5,Abs Gap.5 Dev.10,Abs Gap.5 Dev.20,Abs Gap.5 Dev.40,Abs Gap.10 Dev.1,Abs Gap.10 Dev.3,Abs Gap.10 Dev.5,Abs Gap.10 Dev.10,Abs Gap.10 Dev.20,Abs Gap.10 Dev.40,Abs Gap.20 Dev.1,Abs Gap.20 Dev.3,Abs Gap.20 Dev.5,Abs Gap.20 Dev.10,Abs Gap.20 Dev.20,Abs Gap.20 Dev.40,Abs Gap.40 Dev.1,Abs Gap.40 Dev.3,Abs Gap.40 Dev.5,Abs Gap.40 Dev.10,Abs Gap.40 Dev.20,Abs Gap.40 Dev.40,Abs Gap Lvl,Abs Gap.1 Lvl.1,Abs Gap.1 Lvl.3,Abs Gap.1 Lvl.5,Abs Gap.1 Lvl.10,Abs Gap.1 Lvl.20,Abs Gap.1 Lvl.40,Abs Gap.3 Lvl.1,Abs Gap.3 Lvl.3,Abs Gap.3 Lvl.5,Abs Gap.3 Lvl.10,Abs Gap.3 Lvl.20,Abs Gap.3 Lvl.40,Abs Gap.5 Lvl.1,Abs Gap.5 Lvl.3,Abs Gap.5 Lvl.5,Abs Gap.5 Lvl.10,Abs Gap.5 Lvl.20,Abs Gap.5 Lvl.40,Abs Gap.10 Lvl.1,Abs Gap.10 Lvl.3,Abs Gap.10 Lvl.5,Abs Gap.10 Lvl.10,Abs Gap.10 Lvl.20,Abs Gap.10 Lvl.40,Abs Ga

In [28]:
def MAKE_AGGREGATION_KEYS(Src, TOP):
    TLINE = Src['N'].sort_values(ascending=0).unique()
    return pd.DataFrame(it.product(TLINE, TOP), columns=['N', 'Top'])


Agg = MAKE_AGGREGATION_KEYS(Slc, TOP)
Agg

,N,Top
0,-1,5
1,-1,10
2,-1,25
3,-1,50
4,-1,100
...,...,...
268,-39,25
269,-39,50
270,-39,100
271,-39,250


In [29]:
def ADD_WALLET_PERFORMANCE(Agg, Data, TF, pwr, COLS):

    def _filter(Df, TL, TOP, Rtn):
        COND_TLINE = (Df['N'] == TL)
        COND_TOP   = (Df[f'InTop {TOP}'] == 1)
        COND_ALL   = (COND_TLINE & COND_TOP)
        return Df[COND_ALL][Rtn]

    def _mean(S, rnd, alt=nan):
        try:    return S.mean().round(rnd)
        except: return alt


    Agg['Count'] = Agg.apply(lambda x: (
        _filter(Data, TL=x.N, TOP=int(x.Top), Rtn=f'LS Rtn +1{TF}').count()
    ), axis=1)

    Agg['Return'] = Agg.apply(lambda x: (
        _mean(_filter(Data, TL=x.N, TOP=int(x.Top), Rtn=f'LS Rtn +1{TF}'), rnd=2)
    ), axis=1)

    Agg['CAGR']  = _.CAGR(Agg,  'Return', By='Top', stp=(-1), pwr=pwr)
    Agg['Worth'] = _.Worth(Agg, 'Return', By='Top', stp=(-1))
    Agg['Expnt'] = _.Expnt(Agg,  'Worth', By='Top', stp=(-1))

    Agg['Inds']  = ''
    Agg[COLS]    = Agg.apply(lambda x: (
        _mean(_filter(Data, TL=x.N, TOP=int(x.Top), Rtn=COLS), rnd=2)
    ), axis=1)

    return Agg[['N','Top', 'Count','Expnt','Worth','CAGR','Return', 'Inds',*COLS]]


Agg = ADD_WALLET_PERFORMANCE(Agg, DFrame, TF, ANUM, COLS=[IND_1, IND_2, RNK_1, RNK_2])
Agg.head(60)

,N,Top,Count,Expnt,Worth,CAGR,Return,Inds,Cor 1Y,Gap 1Y,High Cor 1Y,High Gap 1Y
0,-1,5,5,-3.9,4.707,-1.96,56.30,,87.00,68.78,15.20,15.40
1,-1,10,11,-2.7,6.211,-1.25,28.98,,83.97,49.74,20.55,27.55
2,-1,25,38,-5.4,3.279,-2.89,25.77,,75.79,35.55,35.68,44.50
3,-1,50,79,-7.5,2.058,-4.07,18.60,,67.32,28.45,50.35,55.44
4,-1,100,156,-7.3,2.126,-3.99,12.58,,49.30,17.19,75.27,71.31
5,-1,250,189,-7.3,2.125,-3.99,8.28,,37.20,9.83,88.28,83.11
6,-1,500,189,-7.3,2.125,-3.99,8.28,,37.20,9.83,88.28,83.11
7,-2,5,5,-5.8,3.011,-3.19,2.37,,96.24,16.04,7.00,28.60
8,-2,10,15,-3.8,4.815,-1.96,4.73,,93.04,16.86,20.20,28.40
9,-2,25,37,-6.4,2.607,-3.57,1.55,,91.16,12.42,27.54,40.14


In [32]:
def FINAL_RESULTS(Agg, TOP, COLS):

    Top = pd.DataFrame({ 'Top':TOP })

    Top['Count'] = Agg.groupby('Top').apply(lambda By: (
        By['Count'].mean() 
    )).reset_index(drop=1) .apply(round)

    Top[['Expnt', 'Worth', 'CAGR']] = Agg.groupby('Top').apply(lambda By: (
        By[['Expnt', 'Worth', 'CAGR']].head(1)
    )).reset_index(drop=1)

    Top['Return'] = Agg.groupby('Top').apply(lambda By: (
        np.round(By['Return'].mean(), 3)
    )).reset_index(drop=1)

    Top['Inds'] = ''
    Top[COLS]   = Agg.groupby('Top').apply(lambda By: (
        np.round(By[COLS].mean(), 3)
    )).reset_index(drop=1)

    return Top

Top = FINAL_RESULTS(Agg, TOP, COLS=[IND_1, IND_2, RNK_1, RNK_2])
Top

,Top,Count,Expnt,Worth,CAGR,Return,Inds,Cor 1Y,Gap 1Y,High Cor 1Y,High Gap 1Y
0,5,6,-3.9,4.707,-1.96,-0.012,,73.747,39.895,11.241,16.871
1,10,12,-2.7,6.211,-1.25,0.043,,65.003,34.509,15.599,19.470
2,25,29,-5.4,3.279,-2.89,-1.746,,55.073,26.095,22.181,25.912
3,50,52,-7.5,2.058,-4.07,-2.964,,45.853,17.846,30.575,32.261
4,100,81,-7.3,2.126,-3.99,-2.904,,38.034,12.409,39.169,40.698
5,250,91,-7.3,2.125,-3.99,-2.909,,34.944,10.391,42.668,44.443
6,500,91,-7.3,2.125,-3.99,-2.909,,34.944,10.391,42.668,44.443
